## Clustering Methods in Python
In this vignette, we will be comparing three different clusterig methods in python. We will be demonstrating this using a data set that deals with user behavior depending on what kind of phone they have.

### Objective:

## Data Description:
The dataset represents user behavior data collected from various mobile devices and includes 12 attributes for 188 users. Key attributes include Device Model, Operating System, and demographic information such as Age and Gender, alongside numerical metrics that measure daily device usage. These numerical metrics include App Usage Time (min/day), Screen On Time (hours/day), Battery Drain (mAh/day), Number of Apps Installed, and Data Usage (MB/day). Additionally, each user is assigned a User Behavior Class, which categorizes their usage patterns into five distinct classes.

The dataset provides a mix of categorical and numerical features, making it suitable for various clustering and classification analyses. The categorical features, such as Device Model and Operating System, allow for insights into behavioral differences across devices and platforms, while the numerical features provide detailed quantitative data on user interaction with their devices. With its diverse set of attributes, this dataset offers opportunities to explore user segmentation, predict user behavior patterns, and analyze correlations between usage habits and demographics.

## Clustering Methods:
Clsutering is a type of unsupervised machine learning that is used to group similar data points into unique clusters. Although the way in which similarity is determinded varies between the different clustering methods, the end goal is still sorting data points in a way thhat maximizes similarity within a cluster while minimizeing similarity between clusters.

Clustering can be used in many contexts as the unsupervised nature of the method means that it is very flexible. Some of the possible uses of clustering methods include fraud detection, crime analysis, and finance.

We will be comparing three different clustering methods today: k means, spectral, and agglomerative.

#### K-Means Clustering:
K-Means clustering is an unsupervised clustering algorithm that in principle is similar to the supervised machine learning method K-Nearest Neighbors. The algorithm works by initializing K amount of clusters based on prior knowledge or other techniques such as the Elbow method or Silhouette. Then, cluster centroids are randomly assigned. This is followed by assigning data points to clusters whose cluster centroid is closest to as measured by Euclidean distance and recalculating the cluster centroids for each cluster. The last step is iterated until the cluster assignments stop changing. 

Although k-means clustering provides an intuitive framework for conducting cluster analysis, it has a few major drawbacks. First, as cluster centroids are randomly selected, different initializations can lead to different final cluster assignments and can converge to a local minimum instead of a global minimum. Also, as K-means requires the number of clusters to be specific beforehand, which is difficult to determind, results can significantly differ. The algorithm also assumes that clusters are spherical with similar sizes, which in turn means that it is also sensitive to outliers. 

Although not suitible for many complex datasets due to the myriad of drawbacks in the algorithm, K-means clustering provides an easy to understand framework to conduct cluster analysis.


#### Spectral Clustering:
Spectral clustering is an advanced clustering technique that leverages the eigenvalues and eigenvectors of a similarity matrix to group data points based on their relationships, making it particularly effective for non-convex or complex datasets. Unlike traditional methods like k-means, spectral clustering identifies clusters by embedding the data in a lower-dimensional space, revealing hidden structures within the dataset.

In the context of our dataset, spectral clustering can be used to group users into behavior-based segments by considering both categorical features, such as Device Model and Operating System, and numerical features like App Usage Time and Screen On Time. By constructing a similarity matrix based on these features, spectral clustering can uncover intricate patterns in user behavior, identifying clusters that may not be immediately apparent with simpler methods. This approach is particularly useful given the mix of categorical and numerical data, as it allows for a holistic understanding of the relationships between user demographics and device usage habits.

#### Agglomerative Clustering:
Lastly, we have agglomerative clustering which is also an unsupervised machine learning algorithim. It is known as the "bottom-up" approach. Each data point is initially labeled as a cluster. It then works recursively to merge and form larger clusters. There are 3 main steps:
- Creating a proximity matrix
- Using a linkage function to group objects into a hierarchical cluster tree
- Merging clusters togther that are in close proximity